# Ion-Site Occupancy Recommender System Demonstration

This notebook guides you through the process of obtaining new ion-site occupancies based on a given OQMD ID that represents the structural prototype of interest, or given a crystal structure file.

As an example, we will delve into the crystal structure prototype of [$\text{CsV}_3\text{Sb}_5$](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.125.247002), a compound that recently piqued interest for its novel properties. The visualization of this unique crystal structure is displayed below:

![kagome_image](etc/kagome_structure.png "Crystal Structure of CsV3Sb5")

Corresponding to this structure, the OQMD entry with the ID: 1514955 can be found at this [link](https://oqmd.org/materials/entry/1514955). Our aim in this notebook is to demonstrate the process of recommending new ion-site occupancies using this structural prototype as a reference.


The classes used to build the Recommender System are defined in `recommender/core.py` file.

In [1]:
from recommender.core import RecommenderSystem, OccupationData
import joblib
from gensim.models import Word2Vec
from pymatgen.core import Element, Structure

### Loading embedding and defining the data location

In this section, we establish the path to the occupancy data that was used to construct the NetworkX graph. This graph can be found in `data/G100.gexf`. Simultaneously, we load the Word2Vec model, which includes the ion and site embeddings.

We also load the Decision Tree classifier to determine the distance threshold applied within the embedding space. 

In [2]:
json_file_path = 'data/occupancy_data.json.gz'

model = Word2Vec.load(f'models/word2vec.model')
embedding = model.wv

clf = joblib.load(f'models/decision_tree.joblib')
distance_threshold = clf.tree_.threshold[0]

Here we are defining ions that we don't desire to be recommended (noble gases and radioctive elements)

In [3]:
ions = ['Xe', 'Kr', 'Ar', 'Ne', 'He', 'I', 'Br', 'Cl', 'F', 'Te', 'Se', 'S', 'O', 'Bi', 'Sb', 'As', 'P', 'N', 'Pb', 'Sn', 'Ge', 'Si', 'C', 'Tl', 'In', 'Ga', 'Al', 'B', 'Hg', 'Cd', 'Zn', 'Au', 'Ag', 'Cu', 'Pt', 'Pd', 'Ni', 'Ir', 'Rh', 'Co', 'Os', 'Ru', 'Fe', 'Re', 'Tc', 'Mn', 'W', 'Mo', 'Cr', 'Ta', 'Nb', 'V', 'Hf', 'Zr', 'Ti', 'Pu', 'Np', 'U', 'Pa', 'Th', 'Ac', 'Lu', 'Yb', 'Tm', 'Er', 'Ho', 'Dy', 'Tb', 'Gd', 'Eu', 'Sm', 'Pm', 'Nd', 'Pr', 'Ce', 'La', 'Y', 'Sc', 'Ba', 'Sr', 'Ca', 'Mg', 'Be', 'Cs', 'Rb', 'K', 'Na', 'Li', 'H']
ion_forbidden_list = [ion for ion in ions if (Element(ion).Z > 83 or 
                                              ion in ['Tc', 'Pm'] or
                                              Element(ion).group == 18)]
ion_forbidden_list 

['Xe', 'Kr', 'Ar', 'Ne', 'He', 'Tc', 'Pu', 'Np', 'U', 'Pa', 'Th', 'Ac', 'Pm']

First, we create a `OccupationData` object with the compressed json file path.

In [4]:
occupation_data = OccupationData(json_file_path)

At this stage, we retrieve an `AnonymousMotif` object for a specific OQMD ID. This object encapsulates all the crystal structure data, as defined by the Anonymous Motif concept outlined in the paper. This structure-centric information is the foundation for generating our ion-site occupancy recommendations.

In [5]:
kagome_id = 1514955
kagome_AM = occupation_data.get_AM_from_OQMD_id(kagome_id)

The `AnonymousMotif` object provides a prototype Pymatgen Structure consisting exclusively of carbon (C) atoms. It also comes with its equivalent site indexes. After receiving the occupancy recommendations, these resources can be utilized for executing ionic substitutions, effectively transforming the prototype into new material structures.

In [6]:
kagome_AM.example_structure

Structure Summary
Lattice
    abc : 5.469073129 5.469073129363581 9.384302298
 angles : 90.0 90.0 120.00000000384938
 volume : 243.08607542522887
      A : 5.469073129 0.0 0.0
      B : -2.734536565 4.736356265 0.0
      C : 0.0 0.0 9.384302298
    pbc : True True True
PeriodicSite: C (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: C (2.7345, 1.5788, 2.4107) [0.6667, 0.3333, 0.2569]
PeriodicSite: C (-0.0000, 3.1576, 2.4107) [0.3333, 0.6667, 0.2569]
PeriodicSite: C (0.0000, 0.0000, 4.6922) [0.0000, 0.0000, 0.5000]
PeriodicSite: C (2.7345, 1.5788, 6.9736) [0.6667, 0.3333, 0.7431]
PeriodicSite: C (-0.0000, 3.1576, 6.9736) [0.3333, 0.6667, 0.7431]
PeriodicSite: C (2.7345, 0.0000, 4.6922) [0.5000, 0.0000, 0.5000]
PeriodicSite: C (-1.3673, 2.3682, 4.6922) [0.0000, 0.5000, 0.5000]
PeriodicSite: C (1.3673, 2.3682, 4.6922) [0.5000, 0.5000, 0.5000]

In [7]:
kagome_AM.equivalent_sites_indexes

[0, 1, 1, 3, 1, 1, 6, 6, 6]

Next, we'll instantiate a `RecommenderSystem` object. This process involves integrating several elements including `OccupationData`, node embeddings, a predefined distance threshold (given by the trained decision tree), and a list of ions that are deemed unsuitable for inclusion (`ion_forbidden_list`).

One can further refine the model's recommendations by setting a smaller distance threshold, which makes the recommendation process more stringent.

In [8]:
rs = RecommenderSystem(occupation_data=occupation_data, 
                       embedding=embedding, 
                       distance_threshold=distance_threshold, 
                       ion_forbidden_list=ion_forbidden_list)

To generate recommendations for this prototype structure, we feed the `AnonymousMotif`—derived from the OQMD ID—into the `get_recommendations_for_AM` method of the `RecommenderSystem`. The output is a dictionary where each key corresponds to an `AMsite` object representing a specific Anonymous Motif site. Each key is associated with a value that comprises a list of recommended ions for that specific site.

The recommendations are expressed as tuples in the format `(ion, ion-site distance, novel occupation)`. Here, `novel occupation` indicates whether the suggested ion-site occupation represents a new variant within the OQMD compound set (the dataset used to construct the recommender system). The recommendations are sorted based on the ion-site distance.

Importantly, the `AMSite` keys display their respective site indexes. These indexes reference specific sites in the `AnonymousMotif.example_structure` where the recommended ions should be substituted, thus providing a clear mapping for potential new compounds.

In [9]:
kagome_recommendations = rs.get_recommendation_for_AM(kagome_AM)
kagome_recommendations

{(191, 9, (1, '6/mmm'), 0), site index: 0: [('Rb',
   0.018323421478271484,
   False),
  ('K', 0.02188342809677124, False),
  ('Cs', 0.02744007110595703, False),
  ('Tl', 0.11811000108718872, False),
  ('Na', 0.13025516271591187, False),
  ('Ba', 0.261111319065094, True)],
 (191, 9, (1, '6/mmm'), 0), site index: 3: [('Sb',
   0.003526031970977783,
   False),
  ('Bi', 0.028451979160308838, False),
  ('As', 0.07530736923217773, False),
  ('P', 0.15748435258865356, False),
  ('Ir', 0.24137479066848755, True),
  ('Pt', 0.28948670625686646, True),
  ('Ta', 0.2916148900985718, True),
  ('Re', 0.3077079653739929, True),
  ('Rh', 0.3191390633583069, True),
  ('Au', 0.3234304189682007, True),
  ('Os', 0.3247630000114441, True)],
 (191, 9, (3, 'mmm'), 0), site index: 6: [('V', 0.03470265865325928, False),
  ('Nb', 0.07285183668136597, False),
  ('Ta', 0.08547395467758179, False),
  ('Mn', 0.08579069375991821, False),
  ('Ti', 0.09940123558044434, True),
  ('Cr', 0.10330325365066528, False),
  ('

Each `AnonymousMotif` site can be individually accessed through the `sites` attribute. This provides the flexibility to obtain recommendations for a specific Anonymous Motif (AM) site. To do this, simply input the site's label into the `RecommenderSystem.get_recommendation_for_site` method. This approach allows for a more targeted exploration of potential ion substitutions.

In [10]:
kagome_AM.sites

[(191, 9, (1, '6/mmm'), 0), site index: 0,
 (191, 9, (1, '6/mmm'), 0), site index: 3,
 (191, 9, (3, 'mmm'), 0), site index: 6,
 (191, 9, (4, '3m'), 0), site index: 1]

In [11]:
C_site_label = kagome_AM.sites[2].label
# top_n defined to get the top 10 recommendations ranked by the ion-site distance
rs.get_recommendation_for_site(C_site_label, top_n=10) 

[('V', 0.03470265865325928, False),
 ('Nb', 0.07285183668136597, False),
 ('Ta', 0.08547395467758179, False),
 ('Mn', 0.08579069375991821, False),
 ('Ti', 0.09940123558044434, True),
 ('Cr', 0.10330325365066528, False),
 ('Mo', 0.1266164779663086, False),
 ('Re', 0.13882142305374146, True),
 ('Fe', 0.15163785219192505, False),
 ('Hf', 0.19332945346832275, True)]

In [12]:
rs.get_recommendation_for_site(C_site_label, only_new=True) 

[('Ti', 0.09940123558044434, True),
 ('Re', 0.13882142305374146, True),
 ('Hf', 0.19332945346832275, True),
 ('Zr', 0.2026851773262024, True),
 ('Os', 0.20608758926391602, True),
 ('Ni', 0.2988765835762024, True),
 ('Sc', 0.29932743310928345, True),
 ('Li', 0.3114704489707947, True),
 ('Mg', 0.31176626682281494, True),
 ('Hg', 0.3180515170097351, True),
 ('Be', 0.3222390413284302, True)]

Analogously, the same can be done for site recommendations given a specific ion

In [13]:
rs.get_recommendation_for_ion('Bi', top_n=10)

[('12_14_2(-1)_2(2)_2(m)_2(m)_2(m)_4(1)_0[2:2(m)]',
  0.0005753636360168457,
  False),
 ('12_8_1(2/m)_1(2/m)_2(m)_2(m)_2(m)_4[2:2(m)]', 0.0006464719772338867, False),
 ('12_14_2(-1)_2(2)_2(m)_2(m)_2(m)_4(1)_0[3:2(m)]',
  0.0006899833679199219,
  False),
 ('51_8_2(2/m)_2(mm2)_2(mm2)_2(mm2)_21[2:2(mm2)]',
  0.000743865966796875,
  False),
 ('12_18_1(2/m)_1(2/m)_2(m)_2(m)_2(m)_2(m)_2(m)_2(m)_2(m)_2(m)_0[2:2(m)]',
  0.0008195042610168457,
  False),
 ('12_8_1(2/m)_1(2/m)_2(m)_2(m)_2(m)_1[2:2(m)]', 0.000874638557434082, False),
 ('36_16_2(m)_2(m)_2(m)_2(m)_2(m)_2(m)_2(m)_2(m)_0[0:2(m)]',
  0.0008921623229980469,
  False),
 ('2_20_2(1)_2(1)_2(1)_2(1)_2(1)_2(1)_2(1)_2(1)_2(1)_2(1)_2[1:2(1)]',
  0.0009273290634155273,
  False),
 ('36_16_2(m)_2(m)_2(m)_2(m)_2(m)_2(m)_2(m)_2(m)_0[1:2(m)]',
  0.0011115074157714844,
  False),
 ('12_18_1(2/m)_1(2/m)_2(m)_2(m)_2(m)_2(m)_2(m)_2(m)_2(m)_2(m)_0[3:2(m)]',
  0.0012204647064208984,
  False)]

https://www.mindat.org/min-42796.html
CaAlF5

In [16]:
jakobssonite_structure = Structure.from_file('etc/jakobssonite.cif')
jakobssonite_structure

Structure Summary
Lattice
    abc : 8.712 6.317 7.349
 angles : 90.0 115.04000000000003 90.0
 volume : 366.43014260358336
      A : 7.893181100315509 0.0 -3.687361674371263
      B : -3.868046915106915e-16 6.317 3.868046915106915e-16
      C : 0.0 0.0 7.349
    pbc : True True True
PeriodicSite: Ca (-0.0000, 3.4263, 1.8373) [0.0000, 0.5424, 0.2500]
PeriodicSite: Ca (3.9466, 0.2678, -0.0064) [0.5000, 0.0424, 0.2500]
PeriodicSite: Ca (-0.0000, 2.8907, 5.5118) [0.0000, 0.4576, 0.7500]
PeriodicSite: Ca (3.9466, 6.0492, 3.6681) [0.5000, 0.9576, 0.7500]
PeriodicSite: Al (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Al (3.9466, 3.1585, -1.8437) [0.5000, 0.5000, 0.0000]
PeriodicSite: Al (0.0000, 0.0000, 3.6745) [0.0000, 0.0000, 0.5000]
PeriodicSite: Al (3.9466, 3.1585, 1.8308) [0.5000, 0.5000, 0.5000]
PeriodicSite: F (-0.0000, 5.9519, 1.8373) [0.0000, 0.9422, 0.2500]
PeriodicSite: F (3.9466, 2.7934, -0.0064) [0.5000, 0.4422, 0.2500]
PeriodicSite: F (-0.0000, 0.3651, 5.5118) [

In [17]:
jakobssonite_AM = occupation_data.get_AM_from_structure(jakobssonite_structure)
jakobssonite_AM

(15, 14, [(2, '-1'), (2, '2'), (2, '2'), (4, '1'), (4, '1')], 0)

In [18]:
jakobssonite_recommendations = rs.get_recommendation_for_AM(jakobssonite_AM)
jakobssonite_recommendations

{(15, 14, (2, '-1'), 0), site index: 2: [('Na', 0.03471487760543823, False),
  ('K', 0.04308205842971802, False),
  ('Rb', 0.07844394445419312, False),
  ('Cs', 0.1149492859840393, True),
  ('Tl', 0.1474519968032837, False),
  ('Li', 0.21496635675430298, False),
  ('Ba', 0.3086221218109131, True)],
 (15, 14, (2, '2'), 0), site index: 0: [('B', 0.004493117332458496, False),
  ('C', 0.16621387004852295, True),
  ('Al', 0.3119314908981323, False),
  ('Pt', 0.317658007144928, True),
  ('Si', 0.32328367233276367, True),
  ('Ga', 0.32405978441238403, True),
  ('N', 0.32569217681884766, True)],
 (15, 14, (2, '2'), 0), site index: 9: [('S', 0.012279272079467773, False),
  ('Se', 0.022431671619415283, False),
  ('Te', 0.12491995096206665, False)],
 (15, 14, (4, '1'), 0), site index: 3: [('K', 0.0571560263633728, False),
  ('Na', 0.06441515684127808, False),
  ('Rb', 0.07851743698120117, False),
  ('Cs', 0.1478816270828247, True),
  ('Tl', 0.21432268619537354, True),
  ('Li', 0.22979038953781128